In [3]:
import numpy as np
import math as mt
import numpy.random as rnd
from datetime import datetime
from statsmodels.tsa.arima_process import arma_generate_sample
from statsmodels.tsa.stattools import acf, pacf
from scipy.stats import levy_stable
import os
import json
from itertools import product

entr = 84140175472697798260575148673951316302 #rnd.SeedSequence().entropy
rng = rnd.default_rng(entr)
path0 = os.path.expanduser('~')+'/Python Scripts/ARMA_NN/'
SaveFileName_raw = path0 +'raw.json'
SaveFileName_acf = path0 +'acf.json'
SaveFileName_pacf = path0 +'pacf.json'

MaxP = 3
MaxQ = 3

TimePoints = 80
CovLags = int(min(10 * np.log10(TimePoints), TimePoints - 1))
print(f"CovLags = {CovLags}")

alpha_s = [0.25, 1]
phi_s   = [-0.9+0.4*i for i in range(3)]+[0.1+0.4*i for i in range(3)]
theta_s = [0.1+0.4*i for i in range(3)]
RepRepeats = 1


CovLags = 19


In [4]:
def SaveJ(filename, adata):    
    def np_encoder(object):# json fails to process some(?) numpy types, namely, numpy.Int64
        if isinstance(object, np.generic):
            return object.item()
    with open(filename, 'w') as f:
        json.dump(adata, f, indent=2, default=np_encoder)# list of lists - nested data => readability! : indent=2

def LoadJ(filename):
    with open(filename, 'r') as f:
        adata = json.load(f)
    return adata

def get_coeffs_from_quantity(base_coeffs, quantity):
    delta=(base_coeffs[-1]-base_coeffs[0])/(quantity-1)
    return [base_coeffs[0]+delta*_ for _ in range(quantity)]  

def get_N_from_a_p(a,p):
    return a*(a+1)**(p-1)

def get_coeffs(base_coeffs, MaxOrder, Order):
    aMaxN = get_N_from_a_p(len(base_coeffs),MaxOrder)
    if Order==1:
        cc = [get_coeffs_from_quantity( base_coeffs, aMaxN )]
    else:
        c0 = base_coeffs if MaxOrder==Order else get_coeffs_from_quantity( base_coeffs, mt.floor(len(base_coeffs)**(MaxOrder/Order)) )
        cc = [c0]
        c = [list(set(c0+[0]))]
        for _ in range(1,Order-1):            
            cc = c + cc
        c1 = get_coeffs_from_quantity(base_coeffs, max(aMaxN//get_N_from_a_p(len(c0),Order-1) -1,1))
        cc = [list(set(c1+[0]))] + cc
    return cc

def get_N(a,p,b,q):
    return get_N_from_a_p(a,p)*get_N_from_a_p(b,q)

def single_row(idx, aars,amas, aalpha, adata):
    return [idx, list(aars),list(amas),aalpha,list(adata)]

def repeat_agsl(idx_from, ar_quot, ma_quot, aTimePoints, alpha4levy, aCovLags, reps, toRaw, toACF, toPACF):
    tR = []
    tA = []
    tP = []
    ar_quot = np.r_[1, -ar_quot]
    ma_quot = np.r_[1, ma_quot]
    for i in range(idx_from+1, idx_from+reps+1):
        y = arma_generate_sample(ar_quot, ma_quot, nsample=aTimePoints, distrvs=lambda size: levy_stable.rvs(alpha4levy, 0, size=size, random_state=rng))
        tR.append(single_row(i, ar_quot, ma_quot, alpha4levy, y))
        tA.append(single_row(i, ar_quot, ma_quot, alpha4levy, acf(y,adjusted=False,nlags=aCovLags,qstat=False,fft=False,alpha=None,bartlett_confint=False)))
        tP.append(single_row(i, ar_quot, ma_quot, alpha4levy, pacf(y,nlags=aCovLags,method='ywmle',alpha=None)))        
    toRaw.extend(tR)
    toACF.extend(tA)
    toPACF.extend(tP)
    return i

In [5]:
starttime = datetime.now()
RawData = []
ACF = []
PACF = []
global_idx = 0
for p, q in product(list(range(1,MaxP+1)), list(range(1,MaxQ+1))):
    ar = get_coeffs(phi_s, MaxP, p)
    ar = product(*ar)
    ma = get_coeffs(theta_s, MaxQ, q)
    ma = product(*ma)
    arma = product(alpha_s, ar, ma)
    for alpha, ar, ma in arma:
        if isinstance(ar, tuple):
            ar = np.array([*ar])
        else:
            ar = np.array([ar])
        if isinstance(ma, tuple):
            ma = np.array([*ma])
        else:
            ma = np.array([ma])
        global_idx = repeat_agsl(global_idx, ar, ma, TimePoints, alpha, CovLags, RepRepeats, RawData, ACF, PACF)
#        global_idx +=RepRepeats
print(f"Total instances: {global_idx}")
print('before saves ...')
print('Time elapsed {}'.format(datetime.now() - starttime))
SaveJ(SaveFileName_raw,RawData)
SaveJ(SaveFileName_acf,ACF)
SaveJ(SaveFileName_pacf,PACF)
print('after saves ...')
print('Time elapsed {}'.format(datetime.now() - starttime))

Total instances: 248724
before saves ...
Time elapsed 0:06:12.711172
after saves ...
Time elapsed 0:07:21.486372
